In [1]:
import numpy as np
import pandas as pd

import datetime
import time
import hashlib
import json
import sys

import random


In [4]:
sys.path.append('/Users/eyal42/Work/Entrepreneurship/Ramzgate/Hackathon/NodeNeighborhood/priority/src')


In [6]:
import RandomNetwork
import PriorityList

In [19]:
#################################################################################################################
# Delegator node events:
#        1. creation -  when a node is added to the priority list (or readded after being deleted)
#        2. connection -  when a worker is assigned a 'CNCT' job to the node and the job is completed
#        3. pong - when a node appears in the list of neighbors returned from a 'CNCT' job for a different node
#################################################################################################################

class Node:

    def __init__(self,nid):
        tt=datetime.datetime.now()
        ut=datetime.datetime.timestamp(tt)
        self.id=nid
        self.doc={'id':nid,'loc':None,'neigh':[]}
        self.creation=ut
        self.last_connect=None
        self.last_pong=ut
        self.priority=ut
        self.log={'CNCT':0,'PONG':0}
        return
    
    def loadDoc(self,nodeDoc):
        assert self.id==nodeDoc['id'],"Id mismatch"
        self.doc=nodeDoc
        return
    
    def connect(self):
        tt=datetime.datetime.now()
        ut=datetime.datetime.timestamp(tt)
        self.last_connect=ut
        self.priority=ut
        self.log['CNCT']+=1
        return
        
    def pong(self):
        tt=datetime.datetime.now()
        ut=datetime.datetime.timestamp(tt)
        self.last_pong=ut
        self.priority=0.7*ut+0.3*self.priority  # (ut-self.priority)*0.7+self.priority
        self.log['PONG']+=1
        return
    
    #def __str__(self):
    #    return(json.dumps(self.doc)+'--'+self.priority.__str__()+'--'+self.last_pong.__str__()+'--'+self.last_connect.__str__())
    
    def __str__(self):
        jj={
            'node_id':self.id,
            'priority':self.priority.__str__(),
            'last_pong':self.last_pong.__str__(),
            'last_connect':self.last_connect.__str__(),
            'doc':self.doc
        }
        return(json.dumps(jj,indent=2))

    
    def __json__(self):
        return(self.doc)
    


In [5]:
class PriorityList:
    
    def __init__(self):
        self.plist={}
        return

    def inList(self,nid):
        return(nid in self.plist)

    def insert(self,node):
        self.plist[node.id]=node
        return
    
    def remove(self,nid):
        del(self.plist[nid])
        
    def updateNode(self,nid,update):
        if update=='CNCT':
            self.plist[nid].connect()
        elif update=='PONG':
            self.plist[nid].pong()
        else:
            raise Exception("Unknown update")
        return
        
    def getHead(self):
        tmp={self.plist[nid].priority:self.plist[nid].id for nid in self.plist}
        return(tmp[min(tmp)])

    def getTail(self):
        tmp={self.plist[nid].priority:self.plist[nid].id for nid in self.plist}
        return(tmp[max(tmp)])

    def getNode(self,nid):
        return(self.plist[nid])

    def showPriority(self):
        print('length=',len(self.plist))
        tmp={self.plist[nid].priority:nid for nid in self.plist}
        tmp=dict(sorted(tmp.items()))
        for pp in tmp:
            nid=tmp[pp]
            print(pp,nid,self.plist[nid].log)
        return


In [6]:
nt=RandomNetwork.Network(100,0.001)
pl=PriorityList()
bootstrap='0x00000000000000000000'
#bdoc=nt.query(bootstrap)
pl.insert(Node(bootstrap))
pl.showPriority()

length= 1
1703072551.227449 0x00000000000000000000 {'CNCT': 0, 'PONG': 0}


In [7]:
hid=pl.getHead()

for nid in pl.getNode(hid).doc['neigh']:
    if pl.inList(nid):
        pl.updateNode(nid,'PONG')
    else:
        ndoc=nt.query(nid)
        pl.insert(Node(ndoc))
pl.updateNode(hid,'CNCT')
pl.showPriority()

In [8]:
print(pl.plist['0x00000000000000000000'])

{
  "node_id": "0x00000000000000000000",
  "priority": "1703072554.320406",
  "last_pong": "1703072551.227449",
  "last_connect": "1703072554.320406",
  "doc": {
    "id": "0x00000000000000000000",
    "loc": null,
    "neigh": []
  }
}


In [9]:
class Job:
    
    def __init__(self,node,task):
        self.created_time=datetime.datetime.now()
        self.start_time=None
        self.end_time=None
        self.node=node
        self.task=task
        self.jid=self.__getID__()
        self.status='created'
        
    def __getID__(self):
        string=self.created_time.__str__()+self.node.__str__()+self.task
        m = hashlib.sha256()
        m.update(string.encode('utf-8'))
        return(m.hexdigest())
    
    def __str__(self):
        jj={
            'job_id':self.jid,
            'created_time':self.created_time.__str__(),
            'start_time':self.start_time.__str__(),
            'end_time':self.end_time.__str__(),
            'task':self.task,
            'node':self.node.__json__(),
            'status':self.status
        }
        return(json.dumps(jj,indent=2))


In [10]:
class Controller:
    
    def __init__(self):
        self.pl=PriorityList()
        return


    def initialize(self):
        bootstrap='0x00000000000000000000'
        self.pl.insert(Node(bootstrap))
        return
        
    def delegate(self):
        hid=self.pl.getHead()
        node=self.pl.getNode(hid)        
        self.pl.remove(hid) # possibly change this to a 'pull' or 'pop'
        jb=Job(node,'CNCT')
        return(jb)
        
    def ingest(self,jb):
        self.pl.insert(jb.node)
        for nid in jb.node.doc['neigh']:
            if self.pl.inList(nid):
                self.pl.updateNode(nid,'PONG')
            else:
                self.pl.insert(Node(nid))
        self.pl.updateNode(jb.node.id,'CNCT')
        return


In [11]:
class Worker:

    def __init__(self,nt):
        self.nt=nt
        return

    def executeJob(self,jb):
        jb.start_time=datetime.datetime.now()
        jb.status='started'
        nid=jb.node.id
        ndoc=self.nt.query(nid)
        jb.node.doc=ndoc
        jb.end_time=datetime.datetime.now()
        jb.status='completed'
        return(jb)


In [12]:
## Worker Test
nt=RandomNetwork.Network(100,1)
wrkr=Worker(nt)
nn=Node('0x00000000000000000000')
#nn=Node('0xed86f246f734646d674d')
#print(nn)
jb0=Job(nn,'CNCT')
#print(jb0)
jb1=wrkr.executeJob(jb0)
print(jb1)

{
  "job_id": "995df7604067c0f774f5eeb15847b1ff3fd09b3137cac553b48c5deca3b113a9",
  "created_time": "2023-12-20 06:43:11.893539",
  "start_time": "2023-12-20 06:43:11.893639",
  "end_time": "2023-12-20 06:43:12.276314",
  "task": "CNCT",
  "node": {
    "id": "0x00000000000000000000",
    "loc": "Kandahar",
    "neigh": [
      "0x1d29771d9cecc114fc00",
      "0xff08a3e84597fd970ef0",
      "0x8132507dc3d1ffe9201f",
      "0x6337fd63957907483264",
      "0x867ef638be6118edc08b",
      "0xfca763488b0235715a02",
      "0xcf79b973a2cb2d49af45",
      "0x9faead783db73758aeb3",
      "0x777bba38ce86a1300bc3",
      "0xe54183e2a040e6c09e61",
      "0x2456caf1512365bb4622"
    ]
  },
  "status": "completed"
}


In [13]:
class NtSim:
    def __init__(self,size):
        self.size=size
        self.nt=RandomNetwork.Network(self.size,query_time_scale=0.01)
        self.cntrl=Controller()
        self.wrkr=Worker(self.nt)
        return
    
    def Start(self):
        self.cntrl.initialize()
        return
    
    def Iterate(self):
        jb0=self.cntrl.delegate()
        #print(jb0)
        jb1=self.wrkr.executeJob(jb0)
        #print(jb1)
        self.cntrl.ingest(jb1)
        return


In [14]:
ntsim=NtSim(10000)
ntsim.Start()
for ii in range(500):
    ntsim.Iterate()
    #print(len(ntsim.cntrl.pl.pList))
ntsim.cntrl.pl.showPriority()


length= 5179
1703072642.052332 0xf88f86327f618ec0250d {'CNCT': 0, 'PONG': 0}
1703072642.057724 0x5b0e281496c19db09594 {'CNCT': 0, 'PONG': 1}
1703072642.063118 0x389b4f6ee5bd60bebd9d {'CNCT': 0, 'PONG': 0}
1703072642.063213 0x1e86f923a706f617980b {'CNCT': 0, 'PONG': 0}
1703072642.063276 0x164532aeeb7eb9a810ad {'CNCT': 0, 'PONG': 0}
1703072642.063296 0xb4c96d80854dd27e76d8 {'CNCT': 0, 'PONG': 0}
1703072642.063302 0xcdde17637e8cb1e53978 {'CNCT': 0, 'PONG': 0}
1703072642.063309 0x9ee41fddd3dc4bd420d7 {'CNCT': 0, 'PONG': 0}
1703072642.0640755 0xc5652b04914b88d8744b {'CNCT': 0, 'PONG': 1}
1703072642.068405 0x91dbcea3da3ff0c13258 {'CNCT': 0, 'PONG': 0}
1703072642.068413 0x11779dfa5440180b939a {'CNCT': 0, 'PONG': 0}
1703072642.068513 0x6529ca051e6335a77eb7 {'CNCT': 1, 'PONG': 0}
1703072642.0694318 0x14cb32e5c8fda3639d5d {'CNCT': 0, 'PONG': 1}
1703072642.0717251 0x12854b66bd403cf1eda6 {'CNCT': 0, 'PONG': 1}
1703072642.078405 0xd3063b6a09e5da6ff574 {'CNCT': 0, 'PONG': 1}
1703072642.081942 0xe083

In [15]:
len(ntsim.nt.nt)

6346

In [18]:
nnt=RandomNetwork.Network(20000,1)
len(nnt.nt)

8650